In [ ]:
import openai
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# To run the notebook, create a `.env` file at the root of the repo with the following line:
# OPENAI_API_KEY="<INSERT YOUR KEY HERE>"

load_dotenv()

In [ ]:
initial_messages = [
    {"role": "system", "content": "You are a helpful network engineer."},
    {
        "role": "user",
        "content": "How would you respond to a ModBus request? You must be able to respond to all ModBus function codes.",
    },
]
how_to = openai.chat.completions.create(messages=initial_messages, model="o3-mini")
how_to_content = how_to.choices[0].message.content
print(how_to_content)

In [ ]:
messages = initial_messages + [
    {"role": "assistant", "content": how_to_content},
    {
        "role": "user",
        "content": "Using the guide above write a python function to respond to any ModBus request. Always raise an exception over returning an empty bytes value. The request will be in the format of this example: ```79 56 00 00 00 06 01 01 00 0f 00 16```. Respond with only the Python code.",
    },
]
python_code = openai.chat.completions.create(messages=messages, model="o3-mini")
python_code_content = python_code.choices[0].message.content
print(python_code_content)

In [ ]:
# Make a directory for code generated by the LLM.
llm_code_dir = "../llm-generated-code"
Path(llm_code_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# Prepend the LLM generated code with a comment.
python_code_content = "# THIS CODE WAS AUTOMATICALLY GENERATED, DO NOT MODIFY\n# ruff: noqa\n\n" + python_code_content

with open(Path(llm_code_dir).joinpath("modbus_handler.py"), "w") as f:
    f.write(python_code_content)